<a href="https://colab.research.google.com/github/niahizkia/fcc_machine_learning_project/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585.1 MB 4.1 kB/s 
     |████████████████████████████████| 439 kB 47.6 MB/s 
     |████████████████████████████████| 5.9 MB 40.3 MB/s 
     |████████████████████████████████| 1.7 MB 44.2 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2.11.0-dev20220831


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_data = pd.read_csv('train-data.tsv',sep = '\t', names = ['class','text'])
valid_data = pd.read_csv('valid-data.tsv',sep = '\t', names = ['class','text'])

--2022-09-01 04:52:37--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2022-09-01 04:52:37 (24.4 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-09-01 04:52:38--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_data

,class,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [4]:
#Separate label and text
Y_train = train_data.pop('class')
X_train = train_data['text'].tolist()
Y_test = valid_data.pop('class')
X_test = valid_data['text'].tolist()

In [5]:
#one hot encode
Y_train = Y_train.astype('category')
Y_test = Y_test.astype('category')
Y_train = Y_train.cat.codes
Y_test = Y_test.cat.codes

In [12]:
training_labels_final = np.array(Y_train)
testing_labels_final = np.array(Y_test)

In [13]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [14]:
sequences = tokenizer.texts_to_sequences(X_train)

#### Padding

In [15]:
padded = pad_sequences(sequences,maxlen=max_length, padding=padding_type, 
                       truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,
                               padding=padding_type, truncating=trunc_type)

#### Modelling

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           16000     
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 9606      
                                                                 
 dense_3 (Dense)             (None, 2)                 14        
                                                                 
Total params: 25,620
Trainable params: 25,620
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs = 20
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/20
131/131 [==============================] - 1s 5ms/step - loss: 0.3272 - accuracy: 0.8615 - val_loss: 0.1981 - val_accuracy: 0.8657
Epoch 2/20
131/131 [==============================] - 0s 3ms/step - loss: 0.1598 - accuracy: 0.9332 - val_loss: 0.1477 - val_accuracy: 0.9619
Epoch 3/20
131/131 [==============================] - 1s 4ms/step - loss: 0.1247 - accuracy: 0.9775 - val_loss: 0.1233 - val_accuracy: 0.9741
Epoch 4/20
131/131 [==============================] - 1s 6ms/step - loss: 0.1025 - accuracy: 0.9840 - val_loss: 0.1098 - val_accuracy: 0.9777
Epoch 5/20
131/131 [==============================] - 1s 5ms/step - loss: 0.0775 - accuracy: 0.9895 - val_loss: 0.0583 - val_accuracy: 0.9835
Epoch 6/20
131/131 [==============================] - 1s 5ms/step - loss: 0.0236 - accuracy: 0.9928 - val_loss: 0.0477 - val_accuracy: 0.9849
Epoch 7/20
131/131 [==============================] - 1s 4ms/step - loss: 0.0149 - accuracy: 0.9966 - val_loss: 0.0495 - val_accuracy: 0.9842
Epoch 

In [23]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text=[]):
    text=[text]
    text_sequences = tokenizer.texts_to_sequences(text)
    text_padded = pad_sequences(text_sequences,maxlen=max_length, padding=padding_type, 
                           truncating=trunc_type)

    y_sample = model.predict(text_padded)
    ans = round(y_sample[0,1])   
    
    if ans == 1 :
        answer = "spam"
    elif ans == 0 :
        answer = "ham"
    prediction = [y_sample[0,0], answer]
    print(prediction)
    return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 20ms/step
[0.99998915, 'ham']
[0.99998915, 'ham']


In [24]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 22ms/step
[0.99998915, 'ham']
1/1 [==============================] - 0s 19ms/step
[0.25279173, 'spam']
1/1 [==============================] - 0s 19ms/step
[1.0, 'ham']
1/1 [==============================] - 0s 19ms/step
[0.00048883184, 'spam']
1/1 [==============================] - 0s 19ms/step
[0.00024977847, 'spam']
1/1 [==============================] - 0s 19ms/step
[0.9999932, 'ham']
1/1 [==============================] - 0s 24ms/step
[0.99999964, 'ham']
You passed the challenge. Great job!
